# Preriquisites


In [4]:
# Import pandas for data manipulation
import pandas as pd
import numpy as np


In [3]:
# load datafile and preview first few records
hr_df = pd.read_csv('https://bit.ly/2ODZvLCHRDataset')
hr_df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [6]:
# load glossary dataframe and view the records
glossary_df = pd.read_csv('https://bit.ly/2Wz3sWcGlossary')
glossary_df

,employee_id - Unique ID for employee
0,department - Department of employee
1,region - Region of employment (unordered)
2,education - Education Level
3,gender - Gender of Employee
4,recruitment_channel - Channel of recruitment f...
5,nooftrainings - no of other trainings complete...
6,age - Age of Employee
7,previousyearrating - Employee Rating for the p...
8,lengthofservice - Length of service in years
9,KPIs_met >80% - if Percent of KPIs(Key perform...
